In [21]:
import numpy as np
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import apriori,association_rules

In [22]:
jobs = pd.read_csv("archive\jobs_in_data.csv")
jobs.head()

<>:1: DeprecationWarning: invalid escape sequence '\j'
<>:1: DeprecationWarning: invalid escape sequence '\j'
C:\Users\yaraz\AppData\Local\Temp\ipykernel_11512\3755458116.py:1: DeprecationWarning: invalid escape sequence '\j'
  jobs = pd.read_csv("archive\jobs_in_data.csv")


,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93300,United States,Senior,Full-time,In-person,United States,M


In [11]:
jobs.columns

Index(['work_year', 'job_title', 'job_category', 'salary_currency', 'salary',
       'salary_in_usd', 'employee_residence', 'experience_level',
       'employment_type', 'work_setting', 'company_location', 'company_size'],
      dtype='object')

In [32]:
jobs.job_title.unique()

array(['Data DevOps Engineer', 'Data Architect', 'Data Scientist',
       'Machine Learning Researcher', 'Data Engineer',
       'Machine Learning Engineer', 'Data Analyst', 'Analytics Engineer',
       'Applied Scientist', 'BI Developer',
       'Business Intelligence Engineer', 'Research Scientist',
       'Research Analyst', 'Research Engineer', 'Data Science Engineer',
       'Data Quality Analyst', 'Data Product Manager',
       'Machine Learning Scientist', 'AI Engineer', 'MLOps Engineer',
       'Deep Learning Engineer', 'Data Modeler', 'Data Product Owner',
       'Data Science Consultant', 'Business Intelligence Analyst',
       'AI Developer', 'Data Manager', 'ML Engineer',
       'Data Science Director', 'Head of Data', 'BI Analyst',
       'Data Management Analyst', 'Machine Learning Modeler',
       'Data Specialist', 'BI Data Analyst', 'Data Integration Engineer',
       'Business Intelligence Manager', 'Data Integration Specialist',
       'Data Science Practitioner', 'B

In [59]:
jobs['salary_currency'] = jobs['salary_currency'].str.strip()

In [60]:
jobs.dropna(axis = 0, subset =['salary_in_usd'], inplace = True) 
jobs['salary_in_usd'] = jobs['salary_in_usd'].astype('str')

In [61]:
jobs = jobs[~jobs['salary_in_usd'].str.contains('C')] 

In [36]:
jobs

,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93300,United States,Senior,Full-time,In-person,United States,M
...,...,...,...,...,...,...,...,...,...,...,...,...
9350,2021,Data Specialist,Data Management and Strategy,USD,165000,165000,United States,Senior,Full-time,Remote,United States,L
9351,2020,Data Scientist,Data Science and Research,USD,412000,412000,United States,Senior,Full-time,Remote,United States,L
9352,2021,Principal Data Scientist,Data Science and Research,USD,151000,151000,United States,Mid-level,Full-time,Remote,United States,L
9353,2020,Data Scientist,Data Science and Research,USD,105000,105000,United States,Entry-level,Full-time,Remote,United States,S


In [67]:
basket_Germany = (jobs[jobs['company_location'] =="Germany"] 
          .groupby(['experience_level', 'experience_level'])['salary'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('experience_level'))

basket_United_States = (jobs[jobs['company_location'] =="Germany"] 
          .groupby(['experience_level', 'salary_in_usd'])['salary'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('experience_level'))


basket_Germany

experience_level,Entry-level,Executive,Mid-level,Senior
experience_level,,,,
Entry-level,1197000.0,0.0,0.0,0.0
Executive,0.0,230000.0,0.0,0.0
Mid-level,0.0,0.0,1554619.0,0.0
Senior,0.0,0.0,0.0,3112613.0


In [71]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

basket_encoded = basket_Germany.applymap(hot_encode) 
basket_Germany = basket_encoded 

basket_encoded = basket_United_States.applymap(hot_encode) 
basket_United_States = basket_encoded 